In [ ]:
import numpy as np
from random import randint, random

# Define problem-specific constants
n_rooms = 6
room_capacity = {'classroom': 60, 'large_hall': 120}
n_professors = 10
max_courses_per_professor = 3
max_courses_per_section = 5
n_days_per_week = 5
n_slots_per_day = 4
slot_duration = 80
lab_duration = 180
break_duration = 15
total_sections = 10
total_courses = 20

# Define genetic algorithm parameters
n_pop = 50
n_iter = 3
n_bits = n_rooms * n_days_per_week * n_slots_per_day
r_cross = 3
r_mut = 4.0 / n_bits

# Define room types
rooms = ['classroom'] * 3 + ['large_hall'] * 3

# Define class timetable representation (chromosome)
# 0: No class, 1: Class in a classroom, 2: Class in a large hall
class Timetable:
    def __init__(self, chromosome=None):
        if chromosome is None:
            self.chromosome = [randint(0, 2) for _ in range(n_bits)]
        else:
            self.chromosome = chromosome

    def decode(self):
        timetable = np.zeros((n_rooms, n_days_per_week, n_slots_per_day), dtype=int)
        for idx, gene in enumerate(self.chromosome):
            room_idx = idx // (n_days_per_week * n_slots_per_day)
            day_idx = (idx // n_slots_per_day) % n_days_per_week
            slot_idx = idx % n_slots_per_day
            timetable[room_idx, day_idx, slot_idx] = gene
        return timetable

    def fitness(self, chromosome=None, rf=0):
        if chromosome is None:
            chromosome = self.chromosome
        timetable = self.decode()
        fitness = 0

        # Hard constraints
        # Constraint 1: Classes can only be scheduled in free classrooms
        for day in range(n_days_per_week):
            for slot in range(n_slots_per_day):
                for room in range(n_rooms):
                    if timetable[room, day, slot] != 0:
                        fitness += 1

        # Constraint 2: A classroom should be big enough to accommodate the section
        for room_type, capacity in room_capacity.items():
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    room_indices = [i for i, r in enumerate(rooms) if r == room_type]
                    if sum(timetable[room, day, slot] != 0 for room in room_indices) > capacity:
                        fitness -= 1

        # Constraint 3: A professor should not be assigned two different lectures at the same time
        for professor in range(n_professors):
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    lectures_assigned = sum(1 for room in range(n_rooms) if timetable[room, day, slot] == professor)  # Assuming professor assignments start from 0
                    if lectures_assigned > 1:
                        fitness -= 1

        # Constraint 4: The same section cannot be assigned to two different rooms at the same time
        for section in range(total_sections):  # Define total_sections
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    rooms_assigned = sum(1 for room in range(n_rooms) if timetable[room, day, slot] == section)  # Assuming section assignments start from 0
                    if rooms_assigned > 1:
                        fitness -= 1

        # Constraint 5: A room cannot be assigned for two different sections at the same time
        for room in range(n_rooms):
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    sections_assigned = sum(1 for section in range(total_sections) if timetable[room, day, slot] == section)  # Assuming section assignments start from 0
                    if sections_assigned > 1:
                        fitness -= 1
        # Constraint 6: No professor can teach more than 3 courses
        for professor in range(n_professors):
            courses_taught = sum(1 for room in range(n_rooms) for day in range(n_days_per_week) for slot in range(n_slots_per_day) if timetable[room, day, slot] == professor)  # Assuming professor assignments start from 0
            if courses_taught > 3:
                fitness -= 1

        # Constraint 7: No section can have more than 5 courses in a semester
        for section in range(total_sections):  # Define total_sections
            courses_assigned = sum(1 for room in range(n_rooms) for day in range(n_days_per_week) for slot in range(n_slots_per_day) if timetable[room, day, slot] == section)  # Assuming section assignments start from 0
            if courses_assigned > 5:
                fitness -= 1

        # Constraint 8: Each course would have two lectures per week not on the same or adjacent days
        for course in range(total_courses):  # Define total_courses
            lecture_days = []
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    if timetable[room, day, slot] == course:
                        lecture_days.append(day)
            if len(set(lecture_days)) < 2 or any(abs(lecture_days[i] - lecture_days[i+1]) <= 1 for i in range(len(lecture_days)-1)):
                fitness -= 1
        fitness += rf
        # Constraint 9: Lab lectures should be conducted in two consecutive slots
        for section in range(total_sections):  # Define total_sections
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day-1):
                    if timetable[room, day, slot] == section and timetable[room, day, slot+1] == section:
                        fitness -= 1

        # Constraint 10: 15 mins breaks allowed between consecutive classes
        for day in range(n_days_per_week):
            for slot in range(n_slots_per_day-1):
                if timetable[room, day, slot] != 0 and timetable[room, day, slot+1] != 0:
                    fitness -= 1

        # Soft constraints
        # Constraint 11: All theory classes in the morning, all lab sessions in the afternoon
        for day in range(n_days_per_week):
            for slot in range(n_slots_per_day):
                for room in range(n_rooms):
                    if timetable[room, day, slot] == 2:  # Assuming lab sessions are encoded as 2
                        if slot < n_slots_per_day / 2:  # Morning session
                            fitness -= 1
                    elif timetable[room, day, slot] == 1:  # Assuming theory classes are encoded as 1
                        if slot >= n_slots_per_day / 2:  # Afternoon session
                            fitness -= 1

        # Constraint 12: Minimize the number of floors for scheduled classes
##hard to implement it
        # Constraint 13: A class should be held in the same classroom across the whole week
        for section in range(total_sections):  # Define total_sections
            for room in range(n_rooms):
                consecutive_days = 0
                for day in range(n_days_per_week):
                    if timetable[room, day, slot] == section:
                        consecutive_days += 1
                if consecutive_days < n_days_per_week:
                    fitness -= 1

        # Constraint 14: Prefer longer blocks of continuous teaching time for teachers
        for professor in range(n_professors):
            max_continuous_slots = 0
            continuous_slots = 0
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    if timetable[room, day, slot] == professor:
                        continuous_slots += 1
                    else:
                        max_continuous_slots = max(max_continuous_slots, continuous_slots)
                        continuous_slots = 0
            max_continuous_slots = max(max_continuous_slots, continuous_slots)
            fitness -= max_continuous_slots


        return -fitness  # Return negative fitness since we want to minimize conflicts


# Genetic Algorithm Operations
def selection(pop, scores, k=3):
    selection_ix = randint(0, len(pop) - 1)
    for _ in range(k - 1):
        ix = randint(0, len(pop) - 1)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]




def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if random() < r_cross:
        pt1 = randint(1, len(p1)-2)
        pt2 = randint(pt1, len(p1)-1)
        c1[pt1:pt2], c2[pt1:pt2] = p2[pt1:pt2], p1[pt1:pt2]
    return [c1, c2]



def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if random() < r_mut:
            bitstring[i] = 1 - bitstring[i]


# Main Genetic Algorithm function
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    pop = [Timetable() for _ in range(n_pop)]
    best, best_eval = pop[0], objective(pop[0].chromosome)
    for gen in range(n_iter):
        rf = gen + 20
        scores = [objective(c.chromosome, rf) for c in pop]
        # Print generation information
        print("Generation %d:" % gen)

        # Find and print best timetable in the generation
        min_score_idx = scores.index(min(scores))
        print("Best timetable (fitness: %d):" % scores[min_score_idx])
        best_timetable = pop[min_score_idx]
        for room in range(n_rooms):
            for day in range(n_days_per_week):
                for slot in range(n_slots_per_day):
                    class_type = best_timetable.decode()[room, day, slot]
                    if class_type == 1:
                        print("Room %d, Day %d, Slot %d: Class in classroom" % (room+1, day+1, slot+1))
                    elif class_type == 2:
                        print("Room %d, Day %d, Slot %d: Class in large hall" % (room+1, day+1, slot+1))
                    else:
                        print("Room %d, Day %d, Slot %d: No class" % (room+1, day+1, slot+1))

        # Select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = list()
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            for c in crossover(p1.chromosome, p2.chromosome, r_cross):
                mutation(c, r_mut)
                children.append(Timetable(c))
        pop = children
    return [best, best_eval]


if __name__ == '__main__':
    best_timetable, best_fitness = genetic_algorithm(Timetable().fitness, n_bits, n_iter, n_pop, r_cross, r_mut)
    print('Done!')


Generation 0:
Best timetable (fitness: 111):
Room 1, Day 1, Slot 1: Class in classroom
Room 1, Day 1, Slot 2: Class in classroom
Room 1, Day 1, Slot 3: No class
Room 1, Day 1, Slot 4: Class in large hall
Room 1, Day 2, Slot 1: Class in large hall
Room 1, Day 2, Slot 2: No class
Room 1, Day 2, Slot 3: No class
Room 1, Day 2, Slot 4: Class in classroom
Room 1, Day 3, Slot 1: No class
Room 1, Day 3, Slot 2: Class in classroom
Room 1, Day 3, Slot 3: No class
Room 1, Day 3, Slot 4: Class in classroom
Room 1, Day 4, Slot 1: No class
Room 1, Day 4, Slot 2: Class in classroom
Room 1, Day 4, Slot 3: Class in large hall
Room 1, Day 4, Slot 4: Class in classroom
Room 1, Day 5, Slot 1: Class in classroom
Room 1, Day 5, Slot 2: No class
Room 1, Day 5, Slot 3: Class in classroom
Room 1, Day 5, Slot 4: Class in large hall
Room 2, Day 1, Slot 1: No class
Room 2, Day 1, Slot 2: Class in large hall
Room 2, Day 1, Slot 3: Class in large hall
Room 2, Day 1, Slot 4: Class in large hall
Room 2, Day 2, Slot 